In [1]:
import requests
from bs4 import BeautifulSoup
from lxml import html
import json
import re
import csv
import pandas as pd
from collections import Counter
from tqdm import tqdm

In [ ]:
url = 'https://en.wiktionary.org/wiki/Category:Portuguese_idioms'
dict_of_links = {}
while True:
    page = requests.get(url)
    html = page.text
    soup = BeautifulSoup(html)
    for item in soup.select("div[class='mw-category-generated'] a[href^='/wiki/']"):
        if re.search('(?<=>)(.*)(?=<)', str(item)).group(0) == 'Synonymes de langues en français':
            pass
        else:
            collocation = re.search('(?<=>)(.*)(?=<)', str(item)).group(0)
            link = 'https://en.wiktionary.org'+re.search('(?<=href=")([^"]*)', str(item)).group(0)
            dict_of_links[collocation] = link
        #     print('https://en.wiktionary.org'+re.search('(?<=href=")([^"]*)', str(item)).group(0))
    next_page_available = 0
    for i in soup.find_all('a'):
        if i.string == 'next page':
            url='https://en.wiktionary.org/'+i['href']
            next_page_available = 1
            break
    if not next_page_available:
        break
dict_of_links

In [3]:
del dict_of_links['Portuguese rhetorical questions']
del dict_of_links['Portuguese snowclones']
del dict_of_links['Appendix:Portuguese idioms']

In [7]:
len(dict_of_links)

749

In [2]:
from wiktionaryparser import WiktionaryParser
parser = WiktionaryParser()

In [4]:
list_for_file = []
for collocation in tqdm(dict_of_links):
    try:
        collocation_info = parser.fetch(collocation, 'portuguese')
        if collocation_info!=[]:
            if collocation_info[0]['definitions']!=[]:
                for i in range(len(collocation_info[0]['definitions'])):
                    list_for_collocation = [collocation]
                    if 'partOfSpeech' in collocation_info[0]['definitions'][i]:
                        list_for_collocation.append(collocation_info[0]['definitions'][i]['partOfSpeech'])
                        list_for_collocation.append(collocation_info[0]['definitions'][i]['text'][1:])
                    else:
                        list_for_collocation.append('')
                        list_for_collocation.append(collocation_info[0]['definitions'][i]['text'][1:])
                    list_for_collocation.append(dict_of_links[collocation])
                    list_for_file.append(list_for_collocation)
    except AttributeError:
        pass
list_for_file

In [11]:
with open('Portuguese_idioms.tsv', 'w', newline='') as record_file:
    tsv_writer = csv.writer(record_file, delimiter='\t', lineterminator='\n')
    tsv_writer.writerow(['Collocation', 'Part_of_speech', 'Definition', 'Link'])
    for collocation_info in list_for_file:
        tsv_writer.writerow([collocation_info[0], collocation_info[1], collocation_info[2],\
                             dict_of_links[collocation_info[0]]])

In [ ]:
df = pd.read_csv("Portuguese_idioms.tsv",sep='\t')
def left_align(df):
    left_aligned_df = df.style.set_properties(**{'text-align': 'left'})
    left_aligned_df = left_aligned_df.set_table_styles(
        [dict(selector='th', props=[('text-align', 'left')])]
    )
    return left_aligned_df
pd.options.display.max_colwidth=100
pd.options.display.max_rows=50
# display(left_align(df[['Collocation','Part_of_speech','Definition']].head(500)))
display(left_align(df))

In [47]:
list_of_literal_meanings = []
for collocation in tqdm(df['Collocation']):
    collocation_info = parser.fetch(collocation, 'portugese')
    if collocation_info!=[]:
        if collocation_info[0]['etymology']!=[]:
            if not re.search('(?<=Literally, “)([^”]*)', collocation_info[0]['etymology']):
                list_of_literal_meanings.append('')
            else:
                list_of_literal_meanings.append(re.search('(?<=Literally, “)([^”]*)', \
                                                      collocation_info[0]['etymology']).group(0))
#         else:
#             list_of_literal_meanings.append('')
#     else:
#         list_of_literal_meanings.append('')

100%|█████████████████████████████████████████| 546/546 [03:07<00:00,  2.91it/s]


In [48]:
len(list_of_literal_meanings)

546

In [49]:
df.insert(2, 'Literal meaning', list_of_literal_meanings)
df.head()

,Collocation,Part_of_speech,Literal meaning,Definition,Link
0,a caminho,adverb,,"[""(idiomatic) on the way; on one's way; en route""]",https://en.wiktionary.org/wiki/a_caminho
1,a casa caiu,phrase,the house has collapsed,"['(idiomatic) said after a particularly undesirable, harmful change of events; often, though not...",https://en.wiktionary.org/wiki/a_casa_caiu
2,a contragosto,adverb,,['(idiomatic) against the grain; unwillingly; reluctantly'],https://en.wiktionary.org/wiki/a_contragosto
3,a Deus dará,adjective,,['(idiomatic) disordered'],https://en.wiktionary.org/wiki/a_Deus_dar%C3%A1
4,a fila anda,phrase,the line walks,"['(idiomatic) that ship has sailed (that opportunity has been lost)', '(idiomatic) expresses the...",https://en.wiktionary.org/wiki/a_fila_anda


In [53]:
df.to_csv("port_idioms_wLM.tsv", sep = "\t", index=False)